In [1]:
import numpy as np
import random
import torch
from src.dataloaders import Dataset
from src.models import MatrixFactorizationRMSEModel
from src.trainer import Trainer
from src.metrics import hitratio, ndcg
from joblib import Parallel, delayed

np.random.seed(42)
random.seed(42)
torch.manual_seed(42)

device = "cuda" if torch.cuda.is_available() else "cpu"
# device = 'cpu'
print(f"{device=}")

device='cuda'


In [2]:
class config:
    data_dir = "ml-100k"
    neg_count = 20
    epochs = 50
    batch_size = 2048
    dim = 40


dataset = Dataset(config.data_dir)
dataset.gen_adjacency()
dataset.make_train_test()
print(f"{dataset.train_size=}, {dataset.test_size=}")

metrics = {
    # "HR@1": (hitratio, {"top_n": 1}),
    # "HR@5": (hitratio, {"top_n": 5}),
    "HR@10": (hitratio, {"top_n": 10}),
    # "NDCG@1": (ndcg, {"top_n": 1}),
    # "NDCG@5": (ndcg, {"top_n": 5}),
    "NDCG@10": (ndcg, {"top_n": 10}),
}

dataset.train_size=198114, dataset.test_size=943


In [3]:
# took 5 mins to complete

grid_params = {
    "lr": [0.001, 0.01, 0.1, 1],
    "weight_decay": [0.0001, 0.001, 0.01, 0.1, 1],
}


def search(lr, wd):
    model = MatrixFactorizationRMSEModel(
        dataset.user_count, dataset.item_count, config.dim
    )

    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=wd)

    trainer = Trainer(
        dataset,
        model,
        optimizer,
        metrics,
        epochs=config.epochs,
        batch_size=config.batch_size,
        device=device,
    )

    trainer.train(evaluate=True, verbose=False, progressbar=False)
    best_ndcg, best_ndcg_epoch = float("-inf"), 0
    for i, s in enumerate(trainer.test_log):
        if s["NDCG@10"] > best_ndcg:
            best_ndcg = s["NDCG@10"]
            best_ndcg_epoch = i

    return (best_ndcg, best_ndcg_epoch, (lr, wd))


output = Parallel(n_jobs=4)(
    delayed(search)(lr=lr, wd=wd)
    for lr in grid_params["lr"]
    for wd in grid_params["weight_decay"]
)

In [5]:
sorted_log = sorted(output)
sorted_log.reverse()
print(sorted_log[0])

with open("gridsearch_logs/mfrmse.txt", "w") as f:
    for r in sorted_log:
        f.write(f"{r}\n")

(0.5368545035909735, 40, (0.01, 0.0001))


In [9]:
# took 18 mins to complete

grid_params = {
    "b1": list(np.arange(0.1, 1, 0.1)),
    "b2": list(np.arange(0.1, 1, 0.1)),
}


def search(b1, b2):
    model = MatrixFactorizationRMSEModel(
        dataset.user_count, dataset.item_count, config.dim
    )

    optimizer = torch.optim.Adam(
        model.parameters(),
        lr=sorted_log[0][2][0],
        betas=(b1, b2),
        weight_decay=sorted_log[0][2][1],
    )

    trainer = Trainer(
        dataset,
        model,
        optimizer,
        metrics,
        epochs=config.epochs,
        batch_size=config.batch_size,
        device=device,
    )

    trainer.train(evaluate=True, verbose=False, progressbar=False)
    best_ndcg, best_ndcg_epoch = float("-inf"), 0
    for i, s in enumerate(trainer.test_log):
        if s["NDCG@10"] > best_ndcg:
            best_ndcg = s["NDCG@10"]
            best_ndcg_epoch = i

    return (best_ndcg, best_ndcg_epoch, (b1, b2))


output = Parallel(n_jobs=4)(
    delayed(search)(b1=b1, b2=b2)
    for b1 in grid_params["b1"]
    for b2 in grid_params["b2"]
)

In [10]:
sorted_log = sorted(output)
sorted_log.reverse()
print(sorted_log[0])

with open("gridsearch_logs/mfrmse.txt", "a") as f:
    f.write("\n")
    for r in sorted_log:
        f.write(f"{r}\n")

(0.5364471342127466, 39, (0.9, 0.9))
